In [4]:
import requests

RPC_ENDPOINT = "https://mainnet.helius-rpc.com/?api-key=YOUR_API_KEY"

def token_overview(address):
    """
    Fetch token overview using Helius RPC for metadata and DEX Screener for market data.
    """
    print(f'Getting the token overview for {address}')
    result = {}

    try:
        # 1️⃣ Get token metadata using Helius RPC
        payload_metadata = {
            "jsonrpc": "2.0",
            "id": "getAccountInfo",
            "method": "getAccountInfo",
            "params": [address, {"encoding": "jsonParsed"}],
        }
        metadata_response = requests.post(RPC_ENDPOINT, json=payload_metadata)

        if metadata_response.status_code == 200:
            metadata = metadata_response.json().get("result", {}).get("value", {}).get("data", {}).get("parsed", {})
            if metadata:
                result['decimals'] = metadata.get('info', {}).get('decimals', 9)
                result["freezeAuthority"] = metadata.get("info", {}).get("freezeAuthority")
                result["isInitialized"] = metadata.get("info", {}).get("isInitialized")
                result["mintAuthority"] = metadata.get("info", {}).get("mintAuthority")
                result["supply"] = int(metadata.get("info", {}).get("supply", 0))

        # 2️⃣ Get token security info using `getAsset`
        payload_security = {
            "jsonrpc": "2.0",
            "id": "getAsset",
            "method": "getAsset",
            "params": [address],
        }
        security_response = requests.post(RPC_ENDPOINT, json=payload_security)
        if security_response.status_code == 200:
            security_data = security_response.json().get('result', {})
            result['security'] = security_data

        # 3️⃣ Get market data from DEX Screener
        dex_screener_url = f"https://api.dexscreener.com/latest/dex/tokens/{address}"
        dex_response = requests.get(dex_screener_url)

        # Check if DEX Screener returned valid data
        if dex_response.status_code != 200:
            print(f"❌ Failed to fetch DEX Screener data for {address}. Skipping token.")
            return None
        
        dex_data = dex_response.json().get("pairs", None)

        # If no trading pairs exist, the token is dead → SKIP
        if not dex_data:  
            print(f"❌ No trading data on DEX Screener for {address}. Token is likely dead. Skipping.")
            return None  

        # Extract top market data
        top_market = dex_data[0]  
        result['liquidity'] = float(top_market.get("liquidity", {}).get("usd", 0))
        result['mc'] = float(top_market.get("fdv", 0))
        result['v24USD'] = float(top_market.get("volume", {}).get("h24", 0))
        result['trade1h'] = int(top_market.get("txns", {}).get("h1", {}).get("buys", 0) +
                                top_market.get("txns", {}).get("h1", {}).get("sells", 0))
        result['buy_percentage'] = (float(top_market.get("txns", {}).get("h1", {}).get("buys", 0)) /
                                    max(1, result['trade1h']) * 100)

        return result

    except Exception as e:
        print(f"❌ Error retrieving token overview for {address}: {str(e)}")
        return None

if __name__ == "__main__":
    address = "8cquScdifuTxJSg3Lj5epxPpbKXQpak374CkyoBWW2GS"
    overview = token_overview(address)
    if overview:
        print(overview)

Getting the token overview for 8cquScdifuTxJSg3Lj5epxPpbKXQpak374CkyoBWW2GS
❌ No trading data on DEX Screener for 8cquScdifuTxJSg3Lj5epxPpbKXQpak374CkyoBWW2GS. Token is likely dead. Skipping.


In [ ]:
import time
import re
from pathlib import Path
from some_logging_module import log  # Custom logging module for better logging
from some_utils import get_idea_content, track_token_launches, process_transcript, analyze_token_metrics, save_to_file
from data_models import Strategy


class ResearchPhase:
    """Handles the research phase for a trading idea."""

    @staticmethod
    def process(idea_content: str):
        log("🧪 Phase 1: Research")
        strategy, strategy_name = process_transcript(idea_content)

        if not strategy:
            log("❌ Research phase failed!")
            return None

        log(f"🏷️ Strategy Name: {strategy_name}")
        return strategy, strategy_name


class BacktestPhase:
    """Handles the backtest phase for a trading idea."""

    @staticmethod
    def execute():
        log("\n📈 Phase 2: Backtest")
        mint_address, _ = track_token_launches()
        return mint_address


class PackageCheckPhase:
    """Handles the package check for the backtest code."""

    @staticmethod
    def execute(mint_address):
        log("\n📦 Phase 3: Package Check")
        return analyze_token_metrics(mint_address)


class TradingIdeaProcessor:
    """Main class to process a trading idea from link to final backtest."""

    def __init__(self, link: str):
        self.link = link
        self.strategy_name = None
        self.strategy = None
        self.mint_address = None
        self.package_checked = None

    def process_idea(self):
        """Process the trading idea."""
        log("\n🚀 Moon Dev's RBI Agent Processing New Idea!")
        log("🌟 Let's find some alpha in the chaos!")

        try:
            # Step 1: Extract content from the idea
            idea_content = get_idea_content(self.link)
            if not idea_content:
                log("❌ Failed to extract content from idea!")
                return

            log(f"📄 Extracted content length: {len(idea_content)} characters")

            # Phase 1: Research
            research = ResearchPhase()
            self.strategy, self.strategy_name = research.process(idea_content)

            if not self.strategy:
                return

            # Save strategy to file
            strategy_file = save_to_file(self.strategy_name, self.strategy)

            # Phase 2: Backtest
            backtest = BacktestPhase()
            self.mint_address = backtest.execute()

            # Phase 3: Package Check
            package_check = PackageCheckPhase()
            self.package_checked = package_check.execute(self.mint_address)

            if not self.package_checked:
                log("❌ Package check failed!")
                return

            # Save package check output
            save_to_file(self.strategy_name, self.package_checked, directory=PACKAGE_DIR)

            log("🎉 Mission Accomplished!")
            log(f"🚀 Strategy '{self.strategy_name}' is ready to make it rain! 💸")
            log(f"✨ Final backtest saved.")

        except Exception as e:
            log(f"\n❌ Error processing idea: {str(e)}")
            raise


class Main:
    """Main class to process all trading ideas from a file."""

    def __init__(self, ideas_file: Path):
        self.ideas_file = ideas_file

    def process_ideas(self):
        if not self.ideas_file.exists():
            log("❌ ideas.txt not found! Creating template...")
            self.ideas_file.parent.mkdir(parents=True, exist_ok=True)
            with open(self.ideas_file, "w") as f:
                f.write("# Add your trading ideas here (one per line)\n")
                f.write("# Can be YouTube URLs, PDF links, or text descriptions\n")
            return

        with open(self.ideas_file, "r") as f:
            ideas = [line.strip() for line in f if line.strip() and not line.startswith("#")]

        total_ideas = len(ideas)
        log(f"\n🎯 Found {total_ideas} trading ideas to process")

        for i, idea in enumerate(ideas, 1):
            log(f"\n{'='*100}")
            log(f"🌙 Processing idea {i}/{total_ideas}")
            log(f"📝 Idea content: {idea[:100]}{'...' if len(idea) > 100 else ''}")
            log(f"{'='*100}\n")

            try:
                processor = TradingIdeaProcessor(idea)
                processor.process_idea()

                log(f"\n{'='*100}")
                log(f"✅ Completed idea {i}/{total_ideas}")
                log(f"{'='*100}\n")

                if i < total_ideas:
                    log("😴 Taking a break before next idea...")
                    time.sleep(5)

            except Exception as e:
                log(f"\n❌ Error processing idea {i}: {str(e)}")
                log("🔄 Continuing with next idea...\n")
                continue


def main():
    """Main entry point."""
    ideas_file = DATA_DIR / "ideas.txt"
    main_processor = Main(ideas_file)
    main_processor.process_ideas()


if __name__ == "__main__":
    try:
        log(f"\n🌟 Moon Dev's RBI Agent Starting Up!")
        log(f"🤖 Using Research Model: {RESEARCH_MODEL}")
        log(f"📊 Using Backtest Model: {BACKTEST_MODEL}")
        log(f"🔧 Using Debug Model: {DEBUG_MODEL}")
        main()
    except KeyboardInterrupt:
        log("\n👋 Moon Dev's RBI Agent shutting down gracefully...")
    except Exception as e:
        log(f"\n❌ Fatal error: {str(e)}")


In [ ]:
import time
from pathlib import Path
from some_logging_module import log  # Custom logging module for better logging
from some_utils import get_idea_content, track_token_launches, process_transcript, analyze_token_metrics, save_to_file


class ResearchPhase:
    """Handles the research phase for a trading idea."""

    @staticmethod
    def process(idea_content: str):
        log("🧪 Phase 1: Research")
        strategy, strategy_name = process_transcript(idea_content)

        if not strategy:
            log("❌ Research phase failed!")
            return None

        log(f"🏷️ Strategy Name: {strategy_name}")
        return strategy, strategy_name


class TokenAnalysisPhase:
    """Handles the token analysis phase for a trading idea."""

    @staticmethod
    def execute():
        log("\n🔍 Phase 2: Token Analysis")
        mint_address, _ = track_token_launches()
        return mint_address


class TradingIdeaProcessor:
    """Main class to process a trading idea from link to token analysis."""

    def __init__(self, link: str):
        self.link = link
        self.strategy_name = None
        self.strategy = None
        self.mint_address = None
        self.token_analysis = None

    def process_idea(self):
        """Process the trading idea."""
        log("\n🚀 Moon Dev's RBI Agent Processing New Idea!")
        log("🌟 Let's find some alpha in the chaos!")

        try:
            # Step 1: Extract content from the idea
            idea_content = get_idea_content(self.link)
            if not idea_content:
                log("❌ Failed to extract content from idea!")
                return

            log(f"📄 Extracted content length: {len(idea_content)} characters")

            # Phase 1: Research
            research = ResearchPhase()
            self.strategy, self.strategy_name = research.process(idea_content)

            if not self.strategy:
                return

            # Save strategy to file
            strategy_file = save_to_file(self.strategy_name, self.strategy)

            # Phase 2: Token Analysis
            token_analysis = TokenAnalysisPhase()
            self.mint_address = token_analysis.execute()

            # Token analysis output
            self.token_analysis = analyze_token_metrics(self.mint_address)

            if not self.token_analysis:
                log("❌ Token analysis failed!")
                return

            # Save token analysis output
            save_to_file(self.strategy_name, self.token_analysis, directory=PACKAGE_DIR)

            log("🎉 Mission Accomplished!")
            log(f"🚀 Strategy '{self.strategy_name}' is ready to make it rain! 💸")
            log(f"✨ Final token analysis saved.")

        except Exception as e:
            log(f"\n❌ Error processing idea: {str(e)}")
            raise


class Main:
    """Main class to process all trading ideas from a file."""

    def __init__(self, ideas_file: Path):
        self.ideas_file = ideas_file

    def process_ideas(self):
        if not self.ideas_file.exists():
            log("❌ ideas.txt not found! Creating template...")
            self.ideas_file.parent.mkdir(parents=True, exist_ok=True)
            with open(self.ideas_file, "w") as f:
                f.write("# Add your trading ideas here (one per line)\n")
                f.write("# Can be YouTube URLs, PDF links, or text descriptions\n")
            return

        with open(self.ideas_file, "r") as f:
            ideas = [line.strip() for line in f if line.strip() and not line.startswith("#")]

        total_ideas = len(ideas)
        log(f"\n🎯 Found {total_ideas} trading ideas to process")

        for i, idea in enumerate(ideas, 1):
            log(f"\n{'='*100}")
            log(f"🌙 Processing idea {i}/{total_ideas}")
            log(f"📝 Idea content: {idea[:100]}{'...' if len(idea) > 100 else ''}")
            log(f"{'='*100}\n")

            try:
                processor = TradingIdeaProcessor(idea)
                processor.process_idea()

                log(f"\n{'='*100}")
                log(f"✅ Completed idea {i}/{total_ideas}")
                log(f"{'='*100}\n")

                if i < total_ideas:
                    log("😴 Taking a break before next idea...")
                    time.sleep(5)

            except Exception as e:
                log(f"\n❌ Error processing idea {i}: {str(e)}")
                log("🔄 Continuing with next idea...\n")
                continue


def main():
    """Main entry point."""
    ideas_file = DATA_DIR / "ideas.txt"
    main_processor = Main(ideas_file)
    main_processor.process_ideas()


if __name__ == "__main__":
    try:
        log(f"\n🌟 Moon Dev's RBI Agent Starting Up!")
        log(f"🤖 Using Research Model: {RESEARCH_MODEL}")
        log(f"📊 Using Token Analysis Model: {BACKTEST_MODEL}")
        log(f"🔧 Using Debug Model: {DEBUG_MODEL}")
        main()
    except KeyboardInterrupt:
        log("\n👋 Moon Dev's RBI Agent shutting down gracefully...")
    except Exception as e:
        log(f"\n❌ Fatal error: {str(e)}")


In [2]:
import requests
import os

def get_top_holders_percentage(token_address):
    """
    Calculate the percentage of the token's total supply held by its top 10 holders,
    using Helius RPC endpoints:
      - getTokenLargestAccounts for the largest accounts
      - getAccountInfo for token metadata (total supply)
    
    Returns:
        float: The top 10 holders percentage, or None if data cannot be retrieved.
    """
    # 1️⃣ Get the largest token accounts for the given mint
    payload_largest = {
        "jsonrpc": "2.0",
        "id": "getTokenLargestAccounts",
        "method": "getTokenLargestAccounts",
        "params": [token_address],
    }
    largest_response = requests.post(RPC_ENDPOINT, json=payload_largest)
    if largest_response.status_code != 200:
        print(f"❌ Error fetching largest accounts for {token_address}")
        return None

    largest_data = largest_response.json().get("result", {})
    largest_accounts = largest_data.get("value", [])
    if not largest_accounts:
        print(f"❌ No largest accounts data for {token_address}")
        return None

    # Sum amounts of the top 10 accounts (if available)
    top_10_amount = sum(float(acc.get("amount", 0)) for acc in largest_accounts[:10])

    # 2️⃣ Get token metadata to obtain total supply via getAccountInfo
    payload_metadata = {
        "jsonrpc": "2.0",
        "id": "getAccountInfo",
        "method": "getAccountInfo",
        "params": [token_address, {"encoding": "jsonParsed"}],
    }
    metadata_response = requests.post(RPC_ENDPOINT, json=payload_metadata)
    if metadata_response.status_code != 200:
        print(f"❌ Error fetching metadata for {token_address}")
        return None

    metadata = (
        metadata_response.json()
        .get("result", {})
        .get("value", {})
        .get("data", {})
        .get("parsed", {})
    )
    total_supply = int(metadata.get("info", {}).get("supply", 0))
    if total_supply == 0:
        print(f"❌ Total supply is zero for {token_address}")
        return None

    # Calculate and return the percentage
    return (top_10_amount / total_supply) * 100


result = get_top_holders_percentage("2QUgq6g2XjvRWY9f44mBHneYUWtg2faVEBJpybMm7JAz")

❌ Failed to fetch holder data for 2QUgq6g2XjvRWY9f44mBHneYUWtg2faVEBJpybMm7JAz
